# Practice assignment: Handling categorical data

In this programming assignment, you are going to work with the following dataset from Kaggle:

https://www.kaggle.com/arashnic/hr-analytics-job-change-of-data-scientists

This assignment is graded by your `submission.json`.

The cell below creates a valid `submission.json` file, fill your answers in there. 

You can press "Submit Assignment" at any time to submit partial progress.

In [1]:
%%file submission.json
{
    "q1": 0,
    "q2": 0,
    "q3": 0,
    "q4": 0,
    "q5": 0,
    "q6": 0,
    "q7": 0,
    "q8": 0,
    "q9": 0,
    "q10": 0,
    "q11": 0,
    "q12": 0,
    "q13": 0,
    "q14": 0,
    "q15": 0,
    "q16": "",
    "q17": 0
}

Overwriting submission.json


## Dataset description

Suppose that a company working with Big Data and Data Science wants to hire data scientists among people who have successfully passed some courses conducted by the company. Many people sign up for their training. The company wants to focus of the candidates who really want to work for them after training. Information related to demographics, education, experience is provided by the candidates via a sign-up form.

This dataset is designed to understand the factors that lead a person to leave current job which is useful for HR researches. Based on the provided data, you are going to predict whether a candidate is looking for a job change.

The whole data is divided into train and test parts. Data contains several categorical features – they need to be encoded.

## Feature description

- `enrollee_id`: Unique ID for a candidate
- `city`: City code
- `city_development_index`: Developement index of the city (scaled)
- `gender`: Gender of a candidate
- `relevent_experience`: Relevant experience of a candidate
- `enrolled_university`: Type of University course enrolled in if any
- `education_level`: Education level of a candidate
- `major_discipline`: Education major discipline of a candidate
- `experience`: Candidate total experience in years
- `company_size`: Number of employees in a current employer's company
- `company_type`: Type of a current employer
- `last_new_job`: Difference in years between previous and current jobs
- `training_hours`: training hours completed
- `target`: 0 – Not looking for job change, 1 – Looking for a job change


In [2]:
import numpy as np
import pandas as pd

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

## 1

Before modifying the features, let's study our dataframe a bit. First, call `.dtypes` for `train` dataframe. 

**q1:** How many columns in `train` contain data types different from numbers (ints and floats)?

In [4]:
train.dtypes

enrollee_id                 int64
city                       object
city_development_index    float64
gender                     object
relevent_experience        object
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int64
target                    float64
dtype: object

In [5]:
train.select_dtypes(exclude='number')

,city,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job
0,city_16,NaN,Has relevent experience,no_enrollment,High School,NaN,19,100-500,Pvt Ltd,2
1,city_21,NaN,No relevent experience,no_enrollment,Masters,STEM,2,50-99,Early Stage Startup,1
2,city_71,Male,Has relevent experience,Part time course,Graduate,STEM,>20,NaN,NaN,2
3,city_159,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,<10,Pvt Ltd,>4
4,city_16,Male,Has relevent experience,no_enrollment,Masters,STEM,6,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...
19153,city_16,NaN,Has relevent experience,no_enrollment,NaN,NaN,11,NaN,NaN,1
19154,city_103,NaN,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,>4
19155,city_11,NaN,No relevent experience,Full time course,Graduate,STEM,<1,NaN,NaN,never
19156,city_103,Male,No relevent experience,no_enrollment,Graduate,STEM,15,10000+,Pvt Ltd,2


## 2

**q2:** How many unique caterogies does a feature `'city'` contain in the train data?

In [6]:
train['city'].nunique()

123

## 3

**q3:** How many features in train data contain missing values (NaN)?

In [7]:
train.isnull().sum(axis=0)

enrollee_id                  0
city                         0
city_development_index       0
gender                    4508
relevent_experience          0
enrolled_university        386
education_level            460
major_discipline          2813
experience                  65
company_size              5938
company_type              6140
last_new_job               423
training_hours               0
target                       0
dtype: int64

## 4

Some features have a relatively small number of missing values. For instance, `'experience'` has only 65 missing values in the train data. Replacing these missing values with a special category might introduce bias to the data. However, since the number of missing values is not so big, we might be OK with it.

Replace missing values in `'experience'` feature with a category -1.

**q4:** How many categories does this feature contain in the train data now?

_(hint: you might want to use `fillna` function from `pandas` library in this task, but remember that it's not an in-place function by default. Or you can use `SimpleImputer` from `sklearn`)_

In [8]:
train.experience.fillna(-1, inplace=True)

In [9]:
test.experience.fillna(-1, inplace=True)

In [10]:
train['experience'].nunique()

23

## 5

`'education_level'` is an example of an ordinal feature. Sure, we can define an order on it. For instance, `'High School'` is "bigger" than `'Primary School'`, and `'Phd'` is "bigger" than '`Graduate'`. We can encode this feature with integer numbers in a correct order.

In this task, apply a correct mapping for the values of `'education_level'` feature. The mapping should be the following:

* `'Primary School'` -> 0
* `'High School'` -> 1
* `'Graduate'` -> 2
* `'Masters'` -> 3
* `'Phd'` -> 4

At the same time, impute missing values in `'education_level'` with a new category -1. So another part of the mapping would be:

* `np.nan` -> -1

**q5:** What will be the mean value of this feature in the train data after the encoding? Provide the answer, rounded to the nearest TWO decimal places (e.g. 12.3456789 -> 12.35).

_(hint: you might want to use `map` function from `pandas` in this task)_

In [11]:
map_education = {
    'Primary School': 0,
    'High School': 1,
    'Graduate': 2,
    'Masters': 3,
    'Phd': 4,
    np.nan: -1
}
# your code here

In [12]:
train['education_level'] = train['education_level'].map(map_education)

In [13]:
test['education_level'] = test['education_level'].map(map_education)

In [14]:
train['education_level'].mean()

2.061384278108362

## 6

Feature `'relevent_experience'` is an example of a binary feature. You can also check that it has no missing values, which makes its encoding pretty easy.

Encode this feature with the following mapping:

*   `"No relevent experience"` -> 0
*   `"Has relevent experience"` -> 1

**q6:** What will be the mean value of this feature in the train data after the encoding? Provide the answer, rounded to the nearest TWO decimal places (e.g. 12.3456789 -> 12.35).

In [15]:
map_relevent_experience =  {"No relevent experience": 0,
"Has relevent experience": 1}
train['relevent_experience'] = train['relevent_experience'].map(map_relevent_experience)

In [16]:
test['relevent_experience'] = test['relevent_experience'].map(map_relevent_experience)

In [17]:
train['relevent_experience'].mean()

0.7199081323728991

## 7

In our case, `'gender'` is an example of a nominal feature (notice that it is not a binary feature cause it contains three categories + NaNs). We will use One-Hot encoding to encode it. There are several options how to do it in practice: for example, to use `get_dummies` function from `pandas` or `OneHotEncoder` from `sklearn.preprocessing`. Here, we will go with the first option.

If you want to encode a whole dataset with One-Hot encoding, you can directly pass it into the discussed methods. But here we want to encode only one feature. We suggest to do it in four steps:

1. Obtain a One-Hot encoding dataframe for the feature `'gender'`: apply `pd.get_dummies` to this feature. As a result, you should obtain a dataframe with three features (three different categories for gender). Don't include `'NaN'` feature - it will already be included as the encoding (0, 0, 0).
2. Change the column names of this dataframe, so that a feature `'<category_name>'` will become `'gender-<category_name>'`. Of course, it is not a necessary step in general. However, it probably would be more convenient to work with a full dataframe if we remember that these One-Hot encoded features originally came from the feature `'gender'`. On a technical side, you can perform it by changing `df_gender.columns` list.
3. Concatenate original and new dataframes. You can do it by calling `pd.concat` function. Don't forget to set `'axis'` parameter, so that you concatenate dataframes by columns, not by rows. As a result of this step, you should obtain a new `train` dataframe with three new columns named like `'gender-<category_name>'`.
4. Finally, drop `'gender'` feature because we have already encoded it.

As a result of these four steps, you should obtain a new version of `train` dataframe, with dropped `'gender'` feature and three new features named like `'gender-<category_name>'`. A total number of columns by this time should be equal to 16.

**q7:** What is the total number of zero values which appear in the columns starting with `'gender-'`?

_Example: suppose that you obtain the following dataframe:_

| gender-category1 | gender-category2   | gender-category3|
|------|------|------|
|   0  | 1| 0|
|0|0|0|

_Then the answer for the question above should be 5._



In [18]:
train = pd.concat((train, pd.get_dummies(train['gender'], prefix='gender', prefix_sep='-')), axis=1)

In [19]:
train.drop(columns='gender', inplace=True)

In [20]:
(train[['gender-Female', 'gender-Male',
       'gender-Other'
      ]] == 0).astype(int).sum().sum()

42824

In [21]:
test = pd.concat((test, pd.get_dummies(test['gender'], prefix='gender', prefix_sep='-')), axis=1)

In [22]:
test.drop(columns='gender', inplace=True)

## 8

Perform One-Hot encoding for the feature `'enrolled_university'`, using the similar procedure as in the previous task:

1. Obtain a One-Hot encoding dataframe for `'enrolled_university'`.
2. Rename its columns.
3. Concatenate original and One-Hot encoding dataframes.
4. Drop `'enrolled_university'` column.

A total number of columns by this time should be equal to 18.

**q8:** What is the total number of zero values which appear in the columns starting with `'enrolled_university-'`?

In [23]:
train = pd.concat((train, pd.get_dummies(train['enrolled_university'], prefix='enrolled_university', prefix_sep='-')), axis=1)

In [24]:
test = pd.concat((test, pd.get_dummies(test['enrolled_university'], prefix='enrolled_university', prefix_sep='-')), axis=1)

In [25]:
train.drop(columns='enrolled_university', inplace=True)

In [26]:
test.drop(columns='enrolled_university', inplace=True)

In [27]:
(train[[ 'enrolled_university-Full time course',
       'enrolled_university-Part time course',
       'enrolled_university-no_enrollment']] == 0).astype(int).sum().sum()

38702

## 9

Encode feature `'city'` using frequency encoding. You should map each category `'city_i'` to its count (a total number of observations in `train` with `city == 'city_i'`). Save this mapping, since later you would apply the same mapping to the test data.

As a result of this task, feature `'city'` should be encoded with category counts in `train`. 

**q9:** What will be the mean value of this feature in the train data after the encoding? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).


In [28]:
map_city = train['city'].value_counts()
train['city'] = train['city'].map(map_city)

In [29]:
test['city'] = test['city'].map(map_city)

In [30]:
round(train['city'].mean(), 5)

1709.79643

## 10

Encode feature `'last_new_job'` with target encoding with no modifications. First, impute missing values in this feature with a new category `'-1'`. Then, map each category of `'last_new_job'` to the mean target value of the observations with the corresponding category. Save this mapping, since later you would apply the same mapping to the test data.

**q10:** What will be the maximum value of this feature in the train data after the encoding? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.35).

_(hint: you might want to use `groupby` function from `pandas` in this task)_

In [31]:
train['last_new_job'].fillna('-1', inplace=True)

In [32]:
# your code here
map_last_new_job = train.groupby(['last_new_job'])['target'].mean()
# your code here

In [33]:
train['last_new_job'] = train['last_new_job'].map(map_last_new_job)

In [34]:
test['last_new_job'].fillna('-1', inplace=True)

In [35]:
test['last_new_job'] = test['last_new_job'].map(map_last_new_job)

In [36]:
round(train['last_new_job'].max(), 5)

0.36407

## 11

Encode feature `'experience'` with M-estimate encoding. Map each category of `'experience'` according to the following formula:

$$
\hat{x_{ij}} = \frac{\text{target}\left(j, x_{ij}\right) + m\times y_{\text{mean}}}{\text{count}\left(j, x_{ij}\right) + m}\quad,
$$

where

* $x_{ij}$ is a category being encoded,
* $\hat{x_{ij}}$ is its corresponding M-estimate encoding value,
* $\text{count}\left(j, x_{ij}\right)$ is a total number of times $x_{ij}$ appeared in `train`,
* $\text{target}\left(j, x_{ij}\right)$ is a mean target value of the observations with the corresponding category,
* $m$ is a parameter.

In this task, set $m = 0.5$. 

**q11:** What will be the maximum value of this feature in the train data after the encoding? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

In [37]:
m = 0.5
map_experience = (train.groupby('experience')['target'].sum() + m * train['target'].mean()) / (train.groupby('experience')['target'].count() + m)
# your code here

In [38]:
train['experience'] = train['experience'].map(map_experience)

In [39]:
test['experience'] = test['experience'].map(map_experience)

In [40]:
round(train['experience'].max(), 5)

0.45383

## 12

Encode feature `'major_discipline'` with Leave-One-Out encoding. Remember that this technique is similar to target encoding, but here while computing the encoding for a particular observation, we exclude it from the target encoding formula. Therefore a category $x_{ij}$ for the $i$-th observation will be encoded according to the following formula:

$$
\hat{x_{ij}} = \frac{\text{target}\left(j, x_{ij}\right) - y_i}{\text{count}\left(j, x_{ij}\right) - 1}\quad,
$$

where

* $\hat{x_{ij}}$ is its corresponding M-estimate encoding value,
* $\text{count}\left(j, x_{ij}\right)$ is a total number of times $x_{ij}$ appeared in `train`,
* $\text{target}\left(j, x_{ij}\right)$ is a mean target value of the observations with the corresponding category,
* $y_i$ is a target value of the $i$-th observation

For example, suppose that you have the following train data:

|feature|target|
|-|-|
|A|1|
|A|0|
|B|1|
|B|0|
|A|0|

Then you obtain the following Leave-One-Out encoding:

|feature|feature_loo_encoded|
|-|-|
|A|0.0|
|A|0.5|
|B|0.0|
|B|1.0|
|A|0.5|

It is very important to notice that in this method the same category can be encoded differently for different observations. Thus, after encoding the train part of data, you should create a mapping which will help to encode the test data. In order to do this, simply average train encoding values within each category to obtain the final encoding. For instance, suppose that you obtain the following train dataframe:

|feature|feature_loo_encoded|
|-|-|
|A|0.2|
|A|0.6|
|B|0.3|
|B|0.7|
|A|0.4|

Then, for the test data, you should obtain the following mapping:

* A -> 0.4 (because 0.4 is a mean value of the encoded values for the category A)
* B -> 0.5 (because 0.5 is a mean value of the encoded values for the category B)

Don't impute any missing values in this task. After completing this task, drop the feature `'major_discipline'` and rename `'major_discipline_loo_encoded'` to `'major_discipline'`.

**q12:** What will be the maximum value of the encoding values for `'major_discipline'` for the TEST data in the mapping described above? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

In [41]:
train['major_discipline_loo_encoded'] = (train['major_discipline'].map(train.groupby(['major_discipline'])['target'].sum()) - train['target']) / (train['major_discipline'].map(train.groupby(['major_discipline'])['target'].count()) - 1)
map_major_discipline = train.groupby('major_discipline')['major_discipline_loo_encoded'].mean()
# your code here

In [42]:
test['major_discipline_loo_encoded'] = test['major_discipline'].map(map_major_discipline)

In [43]:
train.drop(columns='major_discipline', inplace=True)
train.rename(columns={'major_discipline_loo_encoded': 'major_discipline'}, inplace=True)

In [44]:
test.drop(columns='major_discipline', inplace=True)
test.rename(columns={'major_discipline_loo_encoded': 'major_discipline'}, inplace=True)

In [45]:
round(test['major_discipline'].max(), 5)

0.26772

## 13

Encode feature `'company_size'` with Catboost encoding. The technique was described in the lecture. Here, for the sake of simplicity, let's use the implementation from `category_encoders` library.

As you may remember, Catboost encoding depends on how the data is ordered. Normally, you should shuffle the data one time or even several times. In this task, we will assume that the data has already been shuffled, so you don't need to shuffle it again.

Take `CatBoostEncoder` and use the default values for all its parameters, except `handle_missing` - set it to `'return_nan'` so that your encoder don't do anything with missing values. Fit it on the `'company_size'` (train data) and `'target'` and transform this feature. Save the encoder - it will be used later to transform this feature in test data.

Don't impute any missing values in this task.

**q13:** What will be the most popular value of this feature in the train data after the encoding? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

In [46]:
! pip install category_encoders

In [47]:
from category_encoders.cat_boost import CatBoostEncoder
cbe = CatBoostEncoder()

In [48]:
train['company_size'] = cbe.fit_transform(train['company_size'], train['target'])

In [73]:
test['company_size'] = cbe.transform(test['company_size'])

In [49]:
round(train['company_size'].mode(), 5)

0    0.24935
dtype: float64

## 14

Encode feature `'company_type'` with Weight of Evidence (WoE) encoding. A category $x_{ij}$ will be encoded according to the following formula:

$$
\hat{x_{ij}} = \ln\left(\frac{\mathbb{P}\left(x_{ij}\mid y=1\right)}{\mathbb{P}\left(x_{ij}\mid y=0\right)}\right)\quad.
$$

Here:

$$
\mathbb{P}\left(x_{ij}\mid y=1\right) = \frac{\text{count}\left(y=1\mid x_{ij}\right)}{\text{count}\left(y=1\right)}
$$
$$
\mathbb{P}\left(x_{ij}\mid y=0\right) = \frac{\text{count}\left(y=0\mid x_{ij}\right)}{\text{count}\left(y=0\right)}
$$

The notation means the following:

* $\text{count}\left(y=1\mid x_{ij}\right)$ denotes the number of observations with the category $x_{ij}$ where the target value is equal to $1$;
* $\text{count}\left(y=0\mid x_{ij}\right)$ denotes the same but for the target value $0$;
* $\text{count}\left(y=1\right)$ denotes the number of observations with the target value equal to $1$;
* $\text{count}\left(y=0\right)$ denotes the same but for the target value $0$.


For example, suppose that you have the following train data:

|feature|target|
|-|-|
|A|1|
|A|0|
|B|1|
|B|0|
|A|0|

Then you obtain the following WoE encoding mapping:

* A -> $\ln\left(\frac{\frac{1}{2}}{\frac{2}{3}}\right) \approx -0.288$ 
* B -> $\ln\left(\frac{\frac{1}{2}}{\frac{1}{3}}\right) \approx 0.405$ 

Don't impute any missing values in this task.

**q14:** What will be the most popular value of this feature in the train data after the encoding? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

In [50]:
inter = train.groupby(['company_type', 'target'], as_index=False)['enrollee_id'].count()

In [51]:
inter['target_sep'] = inter['target'].map(train.groupby('target')['enrollee_id'].count())

In [52]:
inter['p'] = inter['enrollee_id'] / inter['target_sep']

In [53]:
map_company_type = np.log(inter.query('target == 1')[['company_type', 'p']].set_index('company_type') / inter.query('target == 0')[['company_type', 'p']].set_index('company_type')).to_dict()['p']
# your code here

In [54]:
train['company_type'] = train['company_type'].map(map_company_type)

In [55]:
test['company_type'] = test['company_type'].map(map_company_type)

In [56]:
round(train['company_type'].mode(), 5)

0   -0.40878
dtype: float64

## 15

We have encoded all categorical features. Next, we drop `'enrollee_id'` because it is not a representative feature. After this, we split train part of data into the dataframe which contains only features (without target) and the target array.

Before training the models, we should impute the remaining missing values. You might have noticed that we didn't impute missing values for the features `'major_discipline'`, `'company_size'` and `'company_type'`. This is because the number of missing values in these features is relatively big (you can check it yourself). In practice, you might just create a special category (like `'-1'`) for each of these features before the encoding. However, in this task, let's perform the imputation using KNN approach - where we impute missing values by looking at the similar observations.

Import `KNNImputer` from `sklearn`. It works only with the dataset with numbers - this is why we didn't run it before the encoding. Set `n_neighbors=3`, and let other parameters have the default values. Fit it on the train dataframe with features, and then transform it. Notice that after the transformation we will obtain `numpy.array` - make `pandas.DataFrame` out of it with the same columns as before.

Save the KNN imputer - you will need it to process the test data. Check that there are no missing values in the train data anymore.

**q15:** What is the mean value of the `'company_size'` feature in the train data after the imputation? Provide the answer, rounded to the nearest FIVE decimal places (e.g. 12.3456789 -> 12.34568).

In [57]:
X_train = train.drop(['enrollee_id', 'target'], axis=1)
y_train = train['target']

X_train.shape, y_train.shape

((19158, 16), (19158,))

In [58]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=3)

In [59]:
train_data = pd.DataFrame(data=imputer.fit_transform(X_train), columns=X_train.columns)

In [60]:
X_train

,city,city_development_index,relevent_experience,education_level,experience,company_size,company_type,last_new_job,training_hours,gender-Female,gender-Male,gender-Other,enrolled_university-Full time course,enrolled_university-Part time course,enrolled_university-no_enrollment,major_discipline
0,1533,0.910,1,1,0.174465,0.249348,-0.408782,0.241379,20,0,0,0,0,0,1,NaN
1,2702,0.624,0,3,0.331818,0.249348,-0.075476,0.264303,10,0,0,0,0,0,1,0.261542
2,266,0.884,1,2,0.153088,0.249348,NaN,0.241379,6,0,1,0,0,1,0,0.261542
3,94,0.843,1,3,0.153088,0.249348,-0.408782,0.182371,56,0,1,0,0,0,1,0.261611
4,1533,0.910,1,3,0.282059,0.624674,NaN,0.221574,15,0,1,0,0,0,1,0.261611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19153,1533,0.910,1,-1,0.227426,0.405871,NaN,0.264303,4,0,0,0,0,0,1,NaN
19154,4355,0.920,1,2,0.153088,0.405802,NaN,0.182371,56,0,0,0,0,0,1,0.261542
19155,247,0.550,0,2,0.453827,0.405903,NaN,0.301387,12,0,0,0,1,0,0,0.261611
19156,4355,0.920,0,2,0.166241,0.190317,-0.408782,0.241379,16,0,1,0,0,0,1,0.261542


In [61]:
train_data

,city,city_development_index,relevent_experience,education_level,experience,company_size,company_type,last_new_job,training_hours,gender-Female,gender-Male,gender-Other,enrolled_university-Full time course,enrolled_university-Part time course,enrolled_university-no_enrollment,major_discipline
0,1533.0,0.910,1.0,1.0,0.174465,0.249348,-0.408782,0.241379,20.0,0.0,0.0,0.0,0.0,0.0,1.0,0.261611
1,2702.0,0.624,0.0,3.0,0.331818,0.249348,-0.075476,0.264303,10.0,0.0,0.0,0.0,0.0,0.0,1.0,0.261542
2,266.0,0.884,1.0,2.0,0.153088,0.249348,-0.429107,0.241379,6.0,0.0,1.0,0.0,0.0,1.0,0.0,0.261542
3,94.0,0.843,1.0,3.0,0.153088,0.249348,-0.408782,0.182371,56.0,0.0,1.0,0.0,0.0,0.0,1.0,0.261611
4,1533.0,0.910,1.0,3.0,0.282059,0.624674,-0.297680,0.221574,15.0,0.0,1.0,0.0,0.0,0.0,1.0,0.261611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19153,1533.0,0.910,1.0,-1.0,0.227426,0.405871,-0.327249,0.264303,4.0,0.0,0.0,0.0,0.0,0.0,1.0,0.261611
19154,4355.0,0.920,1.0,2.0,0.153088,0.405802,-0.408782,0.182371,56.0,0.0,0.0,0.0,0.0,0.0,1.0,0.261542
19155,247.0,0.550,0.0,2.0,0.453827,0.405903,-0.408782,0.301387,12.0,0.0,0.0,0.0,1.0,0.0,0.0,0.261611
19156,4355.0,0.920,0.0,2.0,0.166241,0.190317,-0.408782,0.241379,16.0,0.0,1.0,0.0,0.0,0.0,1.0,0.261542


In [62]:
round(train_data['company_size'].mean(), 5)

0.25282

## 16

Finally, train a Random Forest classifier from `sklearn` on the train data. Set `n_estimators=500`, `max_depth=8`, `random_state=13`, and let other parameters have the default values. Check feature importances. 

**q16:** What is the name of the most important feature for this model? Provide the name of the feature.

In [63]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=500, max_depth=8, random_state=13)

In [64]:
rf.fit(train_data, y_train)

RandomForestClassifier(max_depth=8, n_estimators=500, random_state=13)

In [65]:
np.argmax(rf.feature_importances_)

15

In [66]:
rf.feature_names_in_[15]

'major_discipline'

## 17

In this last task, process the test data so that it is possible to make Random Forest predictions on for it. Perform the similar operations as for the train data, but remember that now you work with test observations and therefore all operations are in the inference mode.

1. (task 4) Feature `'experience'`: impute missing values by -1. 
2. (task 5) Feature `'education_level'`: perform ordinal encoding mapping.
3. (task 6) Feature `'relevent_experience'`: perform binary mapping.
4. (task 7) Feature `'gender'`: perform One-Hot encoding and obtain three new features starting with `'gender-'`. Drop `'gender'` feature.
5. (task 8) Feature `'enrolled_university'`: perform One-Hot encoding and obtain three new features starting with `'enrolled_university-'`. Drop `'enrolled_university'` feature.
6. (task 9) Feature `'city'`: perform frequency encoding mapping.
7. (task 10) Feature `'last_new_job'`: impute missing values by -1 and perform target encoding mapping.
8. (task 11) Feature `'experience'`: perform M-estimate encoding mapping.
9. (task 12) Feature `'major_discipline'`: perform Leave-One-Out encoding mapping.
10. (task 13) Feature `'company_size'`: perform Catboost encoding mapping.
11. (task 14) Feature `'company_type'`: perform WoE encoding mapping.
12. (task 15) Split `test` into `X_test` (a dataframe with no `'enrollee_id'` and `'target'`) and `y_test` (an array with target values). Impute missing values by using KNN imputer which you used before (now only in a transform mode).

As a result of the operations described above, you should obtain `X_test` which is a `pandas.DataFrame` with a shape (2129, 16). Calculate the predictions of the trained Random Forest model on it. Check the accuracy of the predictions on test data.

Then calculate the predictions of the same model on `X_train` and check the accuracy there. Compare the accuracies on train and test data. Do you notice something? What, in your opinion, caused such difference in the accuracies?

**q17:** As a result of this task, provide the accuracy for the test data, rounded to the nearest TWO decimal places (e.g. 12.3456789 -> 12.35).

In [74]:
X_test = test.drop(['enrollee_id', 'target'], axis=1)
y_test = test['target']

In [76]:
test_data = pd.DataFrame(data=imputer.transform(X_test), columns=X_test.columns)

In [77]:
X_test.shape

(2129, 16)

In [81]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, rf.predict(test_data))

0.7332080789102865